In [29]:
import plotly.express as px
import pandas as pd
import random
import numpy as np
from scipy.special import expit
from sklearn.linear_model import LogisticRegression
import psychoanalyze as pa

In [30]:
min_intensity = -4
max_intensity = 4
n_trials = 100

In [31]:
intensity_choices = list(range(min_intensity, max_intensity + 1))
intensities = [random.choice(intensity_choices) for _ in range(n_trials)]
results = [random.random() <= expit(intensity) for intensity in intensities]
trials = pd.DataFrame(
    {
        "Intensity": intensities,
        "Result": results,
    }
)
points = pa.points.from_trials(trials).reset_index()
points

,Intensity,n,Hits,Hit Rate
0,-4,11,0,0.000000
1,-3,10,1,0.100000
2,-2,9,1,0.111111
3,-1,12,3,0.250000
4,0,13,7,0.538462
5,1,16,13,0.812500
6,2,8,8,1.000000
7,3,9,9,1.000000
8,4,12,12,1.000000


In [36]:
pd.concat(
        [
            points,
            pd.DataFrame(
                {
                    "Intensity": np.linspace(-4, 4, 100),
                    "Hit Rate": expit(np.linspace(-4, 4, 100)),
                }
            ),
        ],
        keys= ["observed", "fit"],
        names=["Source"],

)

Intensity     n  Hits  Hit Rate
Source                                      
observed 0   -4.000000  11.0   0.0  0.000000
         1   -3.000000  10.0   1.0  0.100000
         2   -2.000000   9.0   1.0  0.111111
         3   -1.000000  12.0   3.0  0.250000
         4    0.000000  13.0   7.0  0.538462
...                ...   ...   ...       ...
fit      95   3.676768   NaN   NaN  0.975320
         96   3.757576   NaN   NaN  0.977192
         97   3.838384   NaN   NaN  0.978925
         98   3.919192   NaN   NaN  0.980529
         99   4.000000   NaN   NaN  0.982014

[109 rows x 4 columns]

In [50]:
intensity_choices

[-4, -3, -2, -1, 0, 1, 2, 3, 4]

In [57]:
fits.predict_proba(pd.DataFrame({"Intensity": intensity_choices}))[:,0]

array([0.96677737, 0.92608518, 0.84361411, 0.69903025, 0.5       ,
       0.30096975, 0.15638589, 0.07391482, 0.03322263])

In [24]:

fits = LogisticRegression(fit_intercept=False).fit(trials[["Intensity"]], trials["Result"])

In [71]:
k = 1
intensity_choices = np.array(list(range(min_intensity, max_intensity + 1)))
model_hit_rates = 1 / (1 + np.exp(-k * intensity_choices))

In [72]:
model_hit_rates

array([0.01798621, 0.04742587, 0.11920292, 0.26894142, 0.5       ,
       0.73105858, 0.88079708, 0.95257413, 0.98201379])

In [25]:
fits.coef_

array([[0.84268427]])

In [26]:
fits.intercept_

array([0.])

In [4]:
trials = pd.DataFrame(
        {"Intensity": [0, 1], "Result": [0, 1]}, index=pd.Index([0, 1], name="TrialID")
    )
points = pa.points.from_trials(trials)
points


,n,Hits,Hit Rate
TrialID,,,
0,1,0,0.0
1,1,0,0.0


In [13]:
after_trial_1 = pd.Series([0, None], name="Hit Rate", index=pd.Index([0, 1], name="Intensity"))
after_trial_2 = pd.Series([0,1], name="Hit Rate", index=pd.Index([0,1], name="Intensity"))
df = pd.concat([after_trial_1, after_trial_2], keys=[0, 1, 1], names=["Trial", "Intensity"])
df = df.reset_index()
df

,Trial,Intensity,Hit Rate
0,0,0,0.0
1,0,1,NaN
2,1,0,0.0
3,1,1,1.0


In [12]:

fig = px.line(df, x="Intensity", y="Hit Rate", animation_group="Intensity", animation_frame="Trial")
print(fig)

Figure({
    'data': [{'hovertemplate': 'Trial=0<br>Intensity=%{x}<br>Hit Rate=%{y}<extra></extra>',
              'ids': array([0]),
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([0]),
              'xaxis': 'x',
              'y': array([0]),
              'yaxis': 'y'}],
    'frames': [{'data': [{'hovertemplate': 'Trial=0<br>Intensity=%{x}<br>Hit Rate=%{y}<extra></extra>',
                          'ids': array([0]),
                          'legendgroup': '',
                          'line': {'color': '#636efa', 'dash': 'solid'},
                          'marker': {'symbol': 'circle'},
                          'mode': 'lines',
                          'name': '',
                          'orientation': 